In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib.request import urlopen, quote,URLError, HTTPError
import time

In [2]:
# 解析高德地图API网址信息
def get_latlng_gaode(address,ak):
    ak= ak
    url="http://restapi.amap.com/v3/geocode/geo?key=%s&address=%s"%(ak,address)
    data=requests.get(url)
    contest=data.json()
    return contest

In [3]:
def get_receiver_lnglat_gaode(address):
    start_time = time.time()
    re_List_gaode = []
    for b in address:
#         print(b)
        re_Dict_gaode = {}
        try:
            temp=get_latlng_gaode(b,ak_gaode)
        except requests.exceptions.ConnectionError: # HTTPError
            re_Dict_gaode["re_location_gaode"] = "请求出错"         
        else:
            if ('geocodes' in temp):
                re_Dict_gaode["re_location_gaode"] = temp['geocodes'][0]['location']
                re_Dict_gaode["re_formatted_address_gaode"] = temp['geocodes'][0]['formatted_address']
                re_Dict_gaode["re_province_gaode"] = temp['geocodes'][0]['province']
                re_Dict_gaode["re_country_gaode"] = temp['geocodes'][0]['country']
                re_Dict_gaode["re_city_gaode"] = temp['geocodes'][0]['city']
                re_Dict_gaode["re_district_gaode"] = temp['geocodes'][0]['district']
                re_Dict_gaode["re_level_gaode"] = temp['geocodes'][0]['level']
                re_Dict_gaode["re_township_gaode"] = temp['geocodes'][0]['township']
                re_Dict_gaode["re_adcode_gaode"] = temp['geocodes'][0]['adcode']
                re_Dict_gaode["re_street_gaode"] = temp['geocodes'][0]['street']
                re_Dict_gaode["re_number_gaode"] = temp['geocodes'][0]['number']
            else:
                pass
        re_List_gaode.append(re_Dict_gaode)
    #     time.sleep(1)
    end_time = time.time()
    print("高德总用时：",end_time-start_time)
    return re_List_gaode

In [4]:
def parse_gaode(df,address):
    # 解析高德
    list_gaode = get_receiver_lnglat_gaode(address)
    # df化(高德)
    df_gaode = pd.DataFrame(list_gaode)
    data_gaode = pd.concat([df,df_gaode],axis=1)
    print("data_gaode:" + str(data_gaode.shape))
    return data_gaode

In [5]:
# 解析百度地图API网址信息
def get_latlng_baidu(address,ak):
    url = 'http://api.map.baidu.com/geocoder/v2/'
    output = 'json'#输出数据的格式
    ak = ak
    add = quote(address) #由于本文地址变量为中文，为防止乱码，先用quote进行编码
    uri = url + '?' + 'address=' + add  + '&output=' + output + '&ak=' + ak
    req = urlopen(uri)
    res = req.read().decode()
    temp = json.loads(res)
    return temp

In [6]:
# 获取百度地图API地址信息
def get_receiver_lnglat_baidu(address):
    start_time = time.time()
    re_List_baidu = []
    for b in address:
#         print(b)
        re_Dict_baidu = {}
        try:
            temp = get_latlng_baidu(b,ak_baidu)
        except HTTPError as e:
            print("请求出错")
            pass
        else:
            if ('result' in temp):
                re_Dict_baidu["re_Lng_baidu"] = temp['result']['location']['lng']
                re_Dict_baidu["re_Lat_baidu"] = temp['result']['location']['lat']
                re_Dict_baidu["re_Precise_baidu"] = temp['result']['precise']
                re_Dict_baidu["re_Confidence_baidu"] = temp['result']['confidence']
                re_Dict_baidu["re_Comprehension_baidu"] = temp['result']['comprehension']
                re_Dict_baidu["re_Level_baidu"] = temp['result']['level']
            else:
                pass
        re_List_baidu.append(re_Dict_baidu)
    #     time.sleep(1)
    end_time = time.time()
    print("百度总用时：", end_time - start_time)
    return re_List_baidu

In [7]:
def parse_baidu(df,address):
    # 解析百度
    list_baidu = get_receiver_lnglat_baidu(address)
    # df化(百度)
    df_baidu = pd.DataFrame(list_baidu)
    data_baidu = pd.concat([df,df_baidu],axis=1)
    print("data_baidu:" + str(data_baidu.shape))
    return data_baidu

In [8]:
# ak码
# ak_gaode = 'c71d9eda293d20db64955275557d92d4'
ak_gaode = "1c2682317206ccaa18ec2487bb45d880"
ak_baidu = 'z3KEtliyTGvj0bFudEkz4GO0GN8eQQa5'

In [9]:
# 导入数据
date = "1902"
fileName = "宅急送(承运商).xlsx"
pinyin = "zhaijisong"
mainpath = "G:\\work\\basic\\carriers\\"
inputpath = mainpath + date + "\\"
data = pd.read_excel(inputpath + fileName)

In [10]:
# 导出数据路径
outpath_gaode = mainpath + date + "\\output\\gaode\\"
name_ANSI_gaode = date + "Info_lnglat_" + pinyin + "_gaode.xlsx"
name_UTF8_gaode = date + "Info_lnglat_" + pinyin + "_gaode.csv"

outpath_baidu = mainpath + date + "\\output\\baidu\\"
name_ANSI_baidu = date + "Info_lnglat_" + pinyin + "_baidu.xlsx"
name_UTF8_baidu = date + "Info_lnglat_" + pinyin + "_baidu.csv"


In [11]:
data["承运商"].value_counts()

宅急送    2801
Name: 承运商, dtype: int64

### 选取有效字段

In [12]:
df = data[["所在省","所在市","收货地址","订单编号","承运商","所在县"]]

In [13]:
df["address_all"]= df["所在省"] +"省"+ df["所在市"] + df["收货地址"]

d:\python\envs\jupyterpy36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### 文件分组并检查

In [14]:
print("data:" + str(data.shape))
print("df:" + str(df.shape))

data:(2801, 49)
df:(2801, 7)


In [15]:
data_gaode = parse_gaode(df,df["address_all"])

高德总用时： 129.54740977287292
data_gaode:(2801, 18)


In [16]:
data_baidu = parse_baidu(df,df["address_all"])

百度总用时： 127.09026908874512
data_baidu:(2801, 13)


In [19]:
# 合表
split_gaode = pd.DataFrame((x.split(',') for x in data_gaode['re_location_gaode']), index=data_gaode.index, columns=['re_siteLng_gaode','re_siteLat_gaode'])
data_gaode = pd.merge(data_gaode,split_gaode,left_index=True,right_index=True)


In [20]:
# 检查合并后结构
print("data:" + str(data.shape))
print("df:" + str(df.shape))
print("data_gaode:" + str(data_gaode.shape))
print("data_baidu:" + str(data_baidu.shape))

data:(2801, 49)
df:(2801, 7)
data_gaode:(2801, 20)
data_baidu:(2801, 13)


In [22]:
data_gaode.to_csv(outpath_gaode + name_UTF8_gaode,index=False)
data_baidu.to_csv(outpath_baidu + name_UTF8_baidu,index=False)

In [21]:
data_gaode.head()

,所在省,所在市,收货地址,订单编号,承运商,所在县,address_all,re_adcode_gaode,re_city_gaode,re_country_gaode,re_district_gaode,re_formatted_address_gaode,re_level_gaode,re_location_gaode,re_number_gaode,re_province_gaode,re_street_gaode,re_township_gaode,re_siteLng_gaode,re_siteLat_gaode
0,河南,洛阳市,河南洛阳市嵩县大章镇三人场村,690000000001008319021822064980,宅急送,嵩县,河南省洛阳市河南洛阳市嵩县大章镇三人场村,410325,洛阳市,中国,嵩县,河南省洛阳市嵩县三人场村,村庄,"111.764944,34.172490",[],河南省,[],[],111.764944,34.172490
1,河南,驻马店市,河南驻马店市遂平县石寨铺乡大魏庄村黄庄,690000000001008319021822064963,宅急送,遂平县,河南省驻马店市河南驻马店市遂平县石寨铺乡大魏庄村黄庄,411728,驻马店市,中国,遂平县,河南省驻马店市遂平县大魏庄村,村庄,"114.069985,33.071957",[],河南省,[],[],114.069985,33.071957
2,河南,商丘市,李集镇郭庄乡,690000000001008319021822064920,宅急送,夏邑县,河南省商丘市李集镇郭庄乡,411426,商丘市,中国,夏邑县,河南省商丘市夏邑县李集镇,村庄,"116.055760,34.313369",[],河南省,[],[],116.055760,34.313369
3,河南,新乡市,河南省原阳县齐街镇东川村,690000000001008319021822064703,宅急送,原阳县,河南省新乡市河南省原阳县齐街镇东川村,410725,新乡市,中国,原阳县,河南省新乡市原阳县东川村,村庄,"114.199519,35.070911",[],河南省,[],[],114.199519,35.070911
4,河南,周口市,蔡河东街蔡河安居苑13号楼一单元四楼西户,690000000001008319021822064633,宅急送,淮阳县,河南省周口市蔡河东街蔡河安居苑13号楼一单元四楼西户,411626,周口市,中国,淮阳县,河南省周口市淮阳县蔡河安居苑|13号楼,门牌号,"114.901382,33.746156",[],河南省,[],[],114.901382,33.746156
